In [2]:
from keras.layers import Input, Dense, Embedding, LSTM, Dropout, concatenate
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.utils import plot_model

Using TensorFlow backend.


In [2]:
#ツイートのテキスト読み込み
test = open("extract_tweet.txt", "r", encoding="utf-8")
lines = test.readlines()
test.close()
print(len(lines))

#ラベル読み込み
test = open("label.txt", "r", encoding="utf-8")
label = test.readlines()
test.close()
print(len(label))

10000
10000


In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.utils.np_utils import to_categorical

maxlen = 50
training_samples = 8000 # training data 80 : validation data 20
validation_samples = 1000
test_samples = len(lines) - (training_samples + validation_samples)
max_words = 20000

# word indexを作成
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

word_index = tokenizer.word_index
print("Found {} unique tokens.".format(len(word_index)))

data = pad_sequences(sequences, maxlen=maxlen)

# バイナリの行列に変換
categorical_labels = to_categorical(label)
labels = np.asarray(categorical_labels)

print("Shape of data tensor:{}".format(data.shape))
print("Shape of label tensor:{}".format(labels.shape))

#学習データとテストデータに分割
x_test = data[training_samples + validation_samples: training_samples + validation_samples+test_samples]
y_test = labels[training_samples + validation_samples: training_samples + validation_samples+test_samples]
data = data[:training_samples + validation_samples]
labels = labels[:training_samples + validation_samples]

# 行列をランダムにシャッフルする
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

Found 18881 unique tokens.
Shape of data tensor:(10000, 50)
Shape of label tensor:(10000, 2)


In [3]:
import keras.backend as K
from functools import partial

def normalize_y_pred(y_pred):
    return K.one_hot(K.argmax(y_pred), y_pred.shape[-1])

def class_true_positive(class_label, y_true, y_pred):
    y_pred = normalize_y_pred(y_pred)
    return K.cast(K.equal(y_true[:, class_label] + y_pred[:, class_label], 2), K.floatx())

def class_accuracy(class_label, y_true, y_pred):
    y_pred = normalize_y_pred(y_pred)
    return K.cast(K.equal(y_true[:, class_label], y_pred[:, class_label]),
                  K.floatx())

def class_precision(class_label, y_true, y_pred):
    y_pred = normalize_y_pred(y_pred)
    return K.sum(class_true_positive(class_label, y_true, y_pred)) / (K.sum(y_pred[:, class_label]) + K.epsilon())


def class_recall(class_label, y_true, y_pred):
    return K.sum(class_true_positive(class_label, y_true, y_pred)) / (K.sum(y_true[:, class_label]) + K.epsilon())


def class_f_measure(class_label, y_true, y_pred):
    precision = class_precision(class_label, y_true, y_pred)
    recall = class_recall(class_label, y_true, y_pred)
    return (2 * precision * recall) / (precision + recall + K.epsilon())


def true_positive(y_true, y_pred):
    y_pred = normalize_y_pred(y_pred)
    return K.cast(K.equal(y_true + y_pred, 2),
                  K.floatx())


def micro_precision(y_true, y_pred):
    y_pred = normalize_y_pred(y_pred)
    return K.sum(true_positive(y_true, y_pred)) / (K.sum(y_pred) + K.epsilon())


def micro_recall(y_true, y_pred):
    return K.sum(true_positive(y_true, y_pred)) / (K.sum(y_true) + K.epsilon())


def micro_f_measure(y_true, y_pred):
    precision = micro_precision(y_true, y_pred)
    recall = micro_recall(y_true, y_pred)
    return (2 * precision * recall) / (precision + recall + K.epsilon())


def average_accuracy(y_true, y_pred):
    class_count = y_pred.shape[-1]
    class_acc_list = [class_accuracy(i, y_true, y_pred) for i in range(class_count)]
    class_acc_matrix = K.concatenate(class_acc_list, axis=0)
    return K.mean(class_acc_matrix, axis=0)


def macro_precision(y_true, y_pred):
    class_count = y_pred.shape[-1]
    return K.sum([class_precision(i, y_true, y_pred) for i in range(class_count)]) / K.cast(class_count, K.floatx())


def macro_recall(y_true, y_pred):
    class_count = y_pred.shape[-1]
    return K.sum([class_recall(i, y_true, y_pred) for i in range(class_count)]) / K.cast(class_count, K.floatx())


def macro_f_measure(y_true, y_pred):
    precision = macro_precision(y_true, y_pred)
    recall = macro_recall(y_true, y_pred)
    return (2 * precision * recall) / (precision + recall + K.epsilon())

In [4]:
p_input = Input(shape=(50, ), dtype='int32', name='Input_postText')

em = Embedding(input_dim=20000, output_dim=50, input_length=50, name='Embedding')(p_input)
#d_em = Dropout(0.5)(em)
lstm_out = LSTM(32, name='LSTM')(em)
#d_lstm_out = Dropout(0.5)(lstm_out)
output = Dense(2, activation='softmax', name = 'output')(lstm_out)

model = Model(inputs=p_input, outputs = output)
model.compile(optimizer='Adam', loss='categorical_crossentropy',  metrics=['acc', macro_precision, macro_recall, macro_f_measure])
model.summary()
#plot_model(model, show_shapes=True, show_layer_names=True, to_file='model_image/model1.png')


early_stopping = EarlyStopping(patience=0, verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_postText (InputLayer)  (None, 50)                0         
_________________________________________________________________
Embedding (Embedding)        (None, 50, 50)            1000000   
_________________________________________________________________
LSTM (LSTM)                  (None, 32)                10624     
_________________________________________________________________
output (Dense)               (None, 2)                 66        
Total params: 1,010,690
Trainable params: 1,010,690
Non-trainable params: 0
_________________________________________________________________


In [6]:
history = model.fit(x_train, y_train,
                    epochs=100, 
                    batch_size=100,
                    validation_data=(x_val, y_val),
                    callbacks=[early_stopping])

Train on 8000 samples, validate on 1000 samples
Epoch 1/100
8000/8000 [==============================] - 8s 1ms/step - loss: 0.6170 - acc: 0.6644 - macro_precision: 0.7046 - macro_recall: 0.6581 - macro_f_measure: 0.6797 - val_loss: 0.4991 - val_acc: 0.7600 - val_macro_precision: 0.7582 - val_macro_recall: 0.7587 - val_macro_f_measure: 0.7584
Epoch 2/100
8000/8000 [==============================] - 6s 748us/step - loss: 0.4264 - acc: 0.8109 - macro_precision: 0.8176 - macro_recall: 0.8099 - macro_f_measure: 0.8137 - val_loss: 0.4397 - val_acc: 0.7950 - val_macro_precision: 0.7978 - val_macro_recall: 0.7922 - val_macro_f_measure: 0.7950
Epoch 3/100
8000/8000 [==============================] - 6s 745us/step - loss: 0.3110 - acc: 0.8750 - macro_precision: 0.8757 - macro_recall: 0.8752 - macro_f_measure: 0.8754 - val_loss: 0.4729 - val_acc: 0.7920 - val_macro_precision: 0.7945 - val_macro_recall: 0.7901 - val_macro_f_measure: 0.7923
Epoch 00003: early stopping


In [7]:
loss_and_metrics = model.evaluate(x_test, y_test)
print(loss_and_metrics)

1000/1000 [==============================] - 1s 862us/step
[0.7722242164611817, 0.688, 0.6776848149299621, 0.6131743302345276, 0.6317350049018859]


In [8]:
classes = model.predict(x_test)
np.savetxt('LSTM_predict.csv', classes, delimiter = ',')